<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/video_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/ADSP Capstone')

In [3]:
!pwd

/content/drive/MyDrive/ADSP Capstone


In [4]:
!pip install moviepy opencv-python --quiet

In [35]:
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip, concatenate_videoclips
import cv2
import numpy as np
import requests
import os
from PIL import Image
import proglog

class SilentLogger(proglog.ProgressBarLogger):
    def bars_callback(self, bar, attr, value, old_value=None):
        pass

# Function to download an image from a URL and save it locally
def download_image(url, path):
    response = requests.get(url)
    with open(path, 'wb') as file:
        file.write(response.content)

# URLs of the images to be used as backgrounds
image_url1 = "https://cdn.choosechicago.com/uploads/2021/06/43d322b9-1571-44d9-8c2b-0f04ff0caa7d-1-900x630.jpeg"
image_url2 = "https://cdn.choosechicago.com/uploads/2019/06/Art_on_theMART_80233741-0324-4d9d-b831-9a1a76731390.jpg"

# Determine the file extension from the URL
def get_extension(url):
    return url.split("?")[0].split(".")[-1].lower()

# Paths to save the downloaded images with dynamic extensions
image_path1 = f"background1.{get_extension(image_url1)}"
image_path2 = f"background2.{get_extension(image_url2)}"

# Download the images
download_image(image_url1, image_path1)
download_image(image_url2, image_path2)

# Convert image to PNG if needed
def convert_to_png(image_path):
    img = Image.open(image_path).convert("RGBA")
    png_path = os.path.splitext(image_path)[0] + ".png"
    img.save(png_path, "PNG")
    return png_path

# Check if the images are JPG or JPEG and convert them to PNG if needed
if get_extension(image_url1) in ["jpg", "jpeg"]:
    image_path1 = convert_to_png(image_path1)
if get_extension(image_url2) in ["jpg", "jpeg"]:
    image_path2 = convert_to_png(image_path2)

# Define a function to create a mask for the video frame
def create_mask(frame, lower_bound, upper_bound):
    hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask_inv = cv2.bitwise_not(mask)
    return mask_inv

# Define the function to process each frame
def process_frame(frame):
    lower_bound = np.array([35, 100, 100])
    upper_bound = np.array([85, 255, 255])
    mask = create_mask(frame, lower_bound, upper_bound)
    mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    fg = cv2.bitwise_and(frame, mask_3ch)
    return fg

# Load and trim the base video to 30 seconds
video = VideoFileClip("avatar.mp4").subclip(0, 30)

# Apply the process_frame function to each frame in the video
processed_video = video.fl_image(process_frame)

# Define common video size
video_width, video_height = 640, 360

# Load the background images, resize them, and set their duration
background1 = ImageClip(image_path1).resize((video_width, video_height)).set_duration(10)
background2 = ImageClip(image_path2).resize((video_width, video_height)).set_duration(20)

# Concatenate the two background images to match the 30-second video duration
background = concatenate_videoclips([background1, background2])

# Resize the video to a width of 200 pixels and set its position slightly away from the left edge
processed_video = processed_video.resize(width=200).set_pos((15, "center"))

# Create a composite clip with the background and the resized video
final_video = CompositeVideoClip([background, processed_video])

# Write the final video to a file
# final_video.write_videofile("test.mp4")

final_video.write_videofile("test.mp4", codec="libx264", logger=SilentLogger())

In [36]:
from IPython.display import HTML
from base64 import b64encode

# Function to display video
def display_video(filename):
    video_path = f'/content/drive/MyDrive/ADSP Capstone/{filename}'
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'''
    <video width=600 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    ''')

# Display the video
display_video("test.mp4")